In [13]:
import numpy as np
import itertools
import sympy as sp
import gurobipy as gp
from gurobipy import GRB

# Define the four functions

## Bin(INT), Bin(INT) -> Bin(INT)
def identity(x):
    return x

def swap(x):
    return 1-x

def to0(_):
    return 0

def to1(_):
    return 1


basic_strategies = [identity, swap, to0, to1]
all_strategies = list(itertools.product(basic_strategies, repeat=2))

(<function __main__.swap(x)>, <function __main__.to0(_)>)

In [2]:
def generate_top(u):
    f_u, g_u = all_strategies[u]
    print(f"f_{u} = {f_u.__name__} \ng_{u} = {g_u.__name__}\n")

    # value 1 at point(a,b) = (f_u(x),g_u(y)), zero everywhere else
    def middle_entry(x,y):
        # value 1 at (f_u(x),g_u(y)) zero everywhere else
        middle = np.zeros((2,2), dtype=int)
        middle[f_u(x), g_u(y)] = 1
        return middle

    top_level = np.array([
                [middle_entry(0,0), middle_entry(1,0)], 
                [middle_entry(0,1), middle_entry(1,1)]
                ]) 

    return top_level

In [23]:
generate_top(6)

f_6 = swap 
g_6 = to0



array([[[[0, 0],
         [1, 0]],

        [[1, 0],
         [0, 0]]],


       [[[0, 0],
         [1, 0]],

        [[1, 0],
         [0, 0]]]])

In [3]:
# make it symbolic
tokens = [f"w{i}" for i in range(16)]
symb = sp.symbols(" ".join(tokens))

In [4]:
%%capture

# add strategy matrices
summed_strategies = sum([symb[i]*generate_top(i) for i in range(16)])

In [5]:
summed_strategies

array([[[[w0 + w10 + w2 + w8, w1 + w11 + w3 + w9],
         [w12 + w14 + w4 + w6, w13 + w15 + w5 + w7]],

        [[w10 + w4 + w6 + w8, w11 + w5 + w7 + w9],
         [w0 + w12 + w14 + w2, w1 + w13 + w15 + w3]]],


       [[[w1 + w10 + w2 + w9, w0 + w11 + w3 + w8],
         [w13 + w14 + w5 + w6, w12 + w15 + w4 + w7]],

        [[w10 + w5 + w6 + w9, w11 + w4 + w7 + w8],
         [w1 + w13 + w14 + w2, w0 + w12 + w15 + w3]]]], dtype=object)

In [6]:
random_values = np.random.rand(16)
num_levels = 1000
quantized_values = np.round(random_values * (num_levels - 1)) / (num_levels - 1)
quantized_values = quantized_values.reshape((2, 2, 2, 2))
normalized_values = quantized_values.copy()

for x in range(normalized_values.shape[0]):
    for y in range(normalized_values.shape[1]):
        # Calculate the sum of values for the current x, y coordinates
        sum_values = np.sum(normalized_values[x, y])
        # Normalize the values for the current x, y coordinates
        normalized_values[x, y] /= sum_values

precision = 3  # desired precision
normalized_values = np.round(normalized_values, precision)
error = 4.0 - np.sum(normalized_values)
normalized_values[0, 0, 0, 0] += error
normalized_values[0, 0, 0, 0] += 4 - np.sum(normalized_values)
normalized_values = np.round(normalized_values, precision)
normalized_values

P_AB_giv_XY = {}
for a,b,x,y in itertools.product([0,1], repeat=4):
    P_AB_giv_XY[(a,b,x,y)] = normalized_values[x][y][a][b]
# sum(P_AB_giv_XY.values())

# print sum of all values at different a,b when x,y is fixed from P_AB_giv_XY
for x,y in itertools.product([0,1], repeat=2):
    sum = 0
    for a,b in itertools.product([0,1], repeat=2):
        # print(f"P({a},{b}|{x},{y}) = {P_AB_giv_XY[(a,b,x,y)]}")
        sum += P_AB_giv_XY[(a,b,x,y)]
    print(f"x={x}, y={y}", f"sum = {sum}")
    

x=0, y=0 sum = 1.0010000000000001
x=0, y=1 sum = 0.999
x=1, y=0 sum = 1.0000000000000002
x=1, y=1 sum = 1.0


In [7]:
random_values = np.random.rand(16)
num_levels = 1000
quantized_values = np.round(random_values * (num_levels - 1)) / (num_levels - 1)
quantized_values = quantized_values.reshape((2, 2, 2, 2))
normalized_values = quantized_values.copy()

for x in range(normalized_values.shape[0]):
    for y in range(normalized_values.shape[1]):
        sum_values = np.sum(normalized_values[x, y])
        normalized_values[x, y] /= sum_values
        normalized_values[x, y] = np.round(normalized_values[x, y], 3)

for x in range(normalized_values.shape[0]):
    for y in range(normalized_values.shape[1]):
        sum_values = np.sum(normalized_values[x, y])
        if sum_values != 1.0:
            idx = np.where(normalized_values[x, y] != 0)[0][0]
            normalized_values[x, y, idx] += 1.0 - sum_values
normalized_values

P_AB_giv_XY = {}
for a,b,x,y in itertools.product([0,1], repeat=4):
    P_AB_giv_XY[(a,b,x,y)] = normalized_values[x][y][a][b]
P_AB_giv_XY

{(0, 0, 0, 0): 0.3260000000000001,
 (0, 0, 0, 1): 0.246,
 (0, 0, 1, 0): 0.206,
 (0, 0, 1, 1): 0.39,
 (0, 1, 0, 0): 0.2060000000000001,
 (0, 1, 0, 1): 0.292,
 (0, 1, 1, 0): 0.32,
 (0, 1, 1, 1): 0.38,
 (1, 0, 0, 0): 0.285,
 (1, 0, 0, 1): 0.329,
 (1, 0, 1, 0): 0.28,
 (1, 0, 1, 1): 0.081,
 (1, 1, 0, 0): 0.184,
 (1, 1, 0, 1): 0.134,
 (1, 1, 1, 0): 0.194,
 (1, 1, 1, 1): 0.149}

Found summed strategies matrices for all U!

In [8]:
# declaring Gurobi variable W_i with each lb=0 and ub=1
m = gp.Model()
W = m.addVars(16, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="W")
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


In [9]:
# # P(AB|XY)

# # decalre variable v
# v = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name="v")

# v=0.000001
# def P_AB_giv_XY(A,B, X, Y):
#     if (A % 2) & B == 0 and X*Y == 0:
#         return v*0.5 + (1-v)/4
#     else:
#         return v*(0.5*np.kron((A % 2) & B, X*Y)) + (1-v)/4
    
#     return v*(0.5*kron((A % 2) & B, X*Y)) + (1-v)/4

# m.update()

In [10]:
# decalre variable v
v = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name="v")

def kron2(a, b, x, y):
    if a ^ b == x * y:
        return 1
    else:
        return 0


def kron(a, b, x, y):
    #return 1 if ((a%2)^b) == (x*y) else 0
    return 1 if ((a%2)^b) == (x*y) else 0



def P_AB_giv_XY(a, b, x, y):
    return v*kron(a,b,x,y)/2 + (1-v)/4

m.update()

In [11]:
# summed_strategies[x,y][a,b]       # [x,y][a,b]
# P_AB_giv_XY(a,b,x,y)              # [a,b][x,y]

In [12]:
sym_to_gurobi = {symb[i]: W[i] for i in range(16)}
for x,y, a,b in itertools.product(range(2), repeat=4):
    # summed_strategies returns SymPy expression
    sympy_expr = summed_strategies[x,y][a,b]

    # Convert the SymPy expression to a Gurobi expression
    gurobi_expr = sum(sym_to_gurobi[sym] * coef for sym, coef in sympy_expr.as_coefficients_dict().items())

    # Add the constraint to the model
    m.addConstr(gurobi_expr == P_AB_giv_XY(a,b,x,y), f"P({a},{b}|{x},{y}) == {gurobi_expr} == {P_AB_giv_XY(a,b,x,y)}")

m.update()

TypeError: 'numpy.float64' object is not callable

In [ ]:
constr = m.getConstrs()

In [ ]:
import itertools

## should add constraint to not allow more output cardinality than input

# card_X, card_A -> output
def generate_functions(card_X, card_A):
    # Generate sets X and A    
    X = range(card_X)
    A = range(card_A)

    # Generate all possible functions
    all_functions = list(itertools.product(A, repeat=card_X))

    # Convert each function into a mapping from X to A
    mapping_functions = []
    for func in all_functions:
        mapping = {x: func[x] for x in X}
        mapping_functions.append(mapping)

    print("card_A**card_X:", len(all_functions)) # should be card_A^card_X
    print("num combinations of f&u:", len(all_functions)**2)
    return mapping_functions


In [ ]:
generate_functions(2,2)

card_A**card_X: 4
num combinations of f&u: 16


[{0: 0, 1: 0}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 1, 1: 1}]

## SymPy Expression

In [ ]:
# summed_strategies[x,y][a,b]
# print("P(a,b|x,y)")

for x,y, a,b in itertools.product(range(2), repeat=4):
     print(f"P({a},{b}|{x},{y}) = ", summed_strategies[x,y][a,b], " = ", P_AB_giv_XY(a,b,x,y))

P(0,0|0,0) =  w0 + w10 + w2 + w8  =  0.25 + 0.5 v + -0.25 v
P(0,1|0,0) =  w1 + w11 + w3 + w9  =  0.25 + 0.0 v + -0.25 v
P(1,0|0,0) =  w12 + w14 + w4 + w6  =  0.25 + 0.0 v + -0.25 v
P(1,1|0,0) =  w13 + w15 + w5 + w7  =  0.25 + 0.5 v + -0.25 v
P(0,0|0,1) =  w10 + w4 + w6 + w8  =  0.25 + 0.5 v + -0.25 v
P(0,1|0,1) =  w11 + w5 + w7 + w9  =  0.25 + 0.0 v + -0.25 v
P(1,0|0,1) =  w0 + w12 + w14 + w2  =  0.25 + 0.0 v + -0.25 v
P(1,1|0,1) =  w1 + w13 + w15 + w3  =  0.25 + 0.5 v + -0.25 v
P(0,0|1,0) =  w1 + w10 + w2 + w9  =  0.25 + 0.5 v + -0.25 v
P(0,1|1,0) =  w0 + w11 + w3 + w8  =  0.25 + 0.0 v + -0.25 v
P(1,0|1,0) =  w13 + w14 + w5 + w6  =  0.25 + 0.0 v + -0.25 v
P(1,1|1,0) =  w12 + w15 + w4 + w7  =  0.25 + 0.5 v + -0.25 v
P(0,0|1,1) =  w10 + w5 + w6 + w9  =  0.25 + 0.0 v + -0.25 v
P(0,1|1,1) =  w11 + w4 + w7 + w8  =  0.25 + 0.5 v + -0.25 v
P(1,0|1,1) =  w1 + w13 + w14 + w2  =  0.25 + 0.5 v + -0.25 v
P(1,1|1,1) =  w0 + w12 + w15 + w3  =  0.25 + 0.0 v + -0.25 v


## Gurobi Constraints

In [ ]:
for cons in constr:
    print(cons)

<gurobi.Constr P(0,0|0,0) == W[0] + W[10] + W[2] + W[8] == 0.25 + 0.5 v + -0.25 v>
<gurobi.Constr P(0,1|0,0) == W[1] + W[11] + W[3] + W[9] == 0.25 + 0.0 v + -0.25 v>
<gurobi.Constr P(1,0|0,0) == W[12] + W[14] + W[4] + W[6] == 0.25 + 0.0 v + -0.25 v>
<gurobi.Constr P(1,1|0,0) == W[13] + W[15] + W[5] + W[7] == 0.25 + 0.5 v + -0.25 v>
<gurobi.Constr P(0,0|0,1) == W[10] + W[4] + W[6] + W[8] == 0.25 + 0.5 v + -0.25 v>
<gurobi.Constr P(0,1|0,1) == W[11] + W[5] + W[7] + W[9] == 0.25 + 0.0 v + -0.25 v>
<gurobi.Constr P(1,0|0,1) == W[0] + W[12] + W[14] + W[2] == 0.25 + 0.0 v + -0.25 v>
<gurobi.Constr P(1,1|0,1) == W[1] + W[13] + W[15] + W[3] == 0.25 + 0.5 v + -0.25 v>
<gurobi.Constr P(0,0|1,0) == W[1] + W[10] + W[2] + W[9] == 0.25 + 0.5 v + -0.25 v>
<gurobi.Constr P(0,1|1,0) == W[0] + W[11] + W[3] + W[8] == 0.25 + 0.0 v + -0.25 v>
<gurobi.Constr P(1,0|1,0) == W[13] + W[14] + W[5] + W[6] == 0.25 + 0.0 v + -0.25 v>
<gurobi.Constr P(1,1|1,0) == W[12] + W[15] + W[4] + W[7] == 0.25 + 0.5 v + -0.25 v

In [ ]:
#m.setObjective(v, GRB.MAXIMIZE)
# obj = sum(W[i] for i in range(16))
# m.setObjective(obj, GRB.MAXIMIZE)
m.setObjective(v, GRB.MAXIMIZE)

m.update()

m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 17 columns and 80 nonzeros
Model fingerprint: 0x089297e2
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 3e-01]
Presolve removed 7 rows and 4 columns
Presolve time: 0.00s
Presolved: 9 rows, 13 columns, 41 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.000000e+00   0.000000e+00      0s
       8    5.0000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.000000000e-01


In [ ]:
v

<gurobi.Var v (value 0.5)>